# 多层感知机

In [0]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#tf.Session与tf.InteractiveSession的区别，InteractiveSession在后面的各项操作无须指定Session
sess = tf.InteractiveSession()

In [0]:
in_units = 784 #输入节点数
h1_units = 300 #隐藏层的输出节点数
#隐藏层的权重和偏置
#将权重初始化为截断的正态分布，其标准差为0.1(因为使用的激活函数是ReLU，所以需要正态分布给参数增加一点噪声来打破完全对称并且避免0梯度)
W1 = tf.Variable(tf.truncated_normal([in_units,h1_units],stddev=0.1)) 
b1 = tf.Variable(tf.zeros([h1_units]))
#输出层softmax的权重和偏置，初始化为0，因为sigmoid在0附近最敏感，梯度最大。
W2 = tf.Variable(tf.zeros([h1_units,10]))
b2 = tf.Variable(tf.zeros([10]))

In [0]:
#输入特征的placeholder
x = tf.placeholder(tf.float32, [None,in_units])

#dropout也是计算图的输入；dropout的比率keep_prob(即保留节点的概率)是不一样的，通常在训练时小于1，而预测时则等于1
keep_prob = tf.placeholder(tf.float32)

In [0]:
#1、定义模型结构。首先需要一个隐藏层，为hidden1，接着，调用tf.nn.dropout实现dropout功能，即随机将一部分节点置为0。
#这里的keep_prob参数即为保留数据而不置为0的比例，在训练时应该是小于1的，用于制造随机性，防止过拟合，最后是输出层

hidden1 = tf.nn.relu(tf.matmul(x,W1)+b1) #隐藏层
hidden1_drop = tf.nn.dropout(hidden1, keep_prob) #调用dropout
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2)+b2) #softmax层

In [0]:
#2、定义损失函数和选择优化器来优化loss，这里的损失函数继续使用交叉熵，而优化器使用Adagrad，学习速率设为0.3
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))

In [0]:
#3、训练步骤
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)
tf.global_variables_initializer().run()

for i in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run({x:batch_xs, y_:batch_ys,keep_prob:0.75})

In [0]:
#4、对模型进行准确率评测
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))